## P14
### Part 1

In [ ]:
reactions = []
with open('p14_input.txt') as f:
    for line in f:
        r = line.split('=>')
        left = [rr.strip().split() for rr in r[0].split(',')]
        right = r[1].strip().split()
        left = [[int(t[0]), t[1]] for t in left]
        right = [int(right[0]), right[1]]
        reactions.append((left, right))
reactions[0]

In [ ]:
from collections import defaultdict, Counter

import math

ingredients = {}
check = Counter()
for rl, rr in reactions:
    print(rl, rr)
    gcd = rr[0]
    for reac_quant in [reac[0] for reac in rl]:
        gcd = math.gcd(gcd, reac_quant)
    if gcd > 1:
        ingredients[rr[1]] = [[int(reac[0]/gcd), reac[1]] for reac in rl], [int(rr[0]/gcd), rr[1]]
        print(ingredients[rr[1]])
        ingredients[rr[1]] = rl, rr
    else:
        ingredients[rr[1]] = rl, rr
    

In [ ]:
import math



def run(fuel):
    
    reac_needs = Counter()
    reac_produced = Counter()

    reac_needs['ORE'] = 0
    reac_needs['FUEL'] = fuel

    while sum(need for key, need in reac_needs.items() if key != 'ORE') > 0:
        for reac in ingredients.keys():
            quant = reac_needs[reac]
            if reac == 'ORE' or quant == 0:
                continue

            prod_need = quant - reac_produced[reac]

            if prod_need > 0:
                reaclist, prod = ingredients[reac]

                mult = math.ceil(prod_need / prod[0])
                mult = max(mult, 1)

                reac_produced[reac] += prod[0]*mult
                for reac_amount, reac_name in reaclist:
                    reac_needs[reac_name] += reac_amount*mult

            reac_produced[reac] -= reac_needs[reac]
            reac_needs[reac] = 0

    return reac_needs['ORE']

In [ ]:
run(1)

### Part 2

In [ ]:
ore_stock = 10**12
min_ = run(1)
max_ = int(ore_stock / min_)

for i in range(4900000, 4920000):
    if run(i) > ore_stock:
        print(i)
        break

In [ ]:
run(4906796)

## P13
### Part 1

In [ ]:
with open('p13_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
from collections import Counter, defaultdict, namedtuple

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

h = 24
w = 44

colors = {
    0: (0, 0, 0),
    1: (255, 125, 0),
    2: (125, 255, 0),
    3: (255, 255, 0),
    4: (255, 0, 255)
}

Point = namedtuple('Point', ['x', 'y'])

def intersect(ap1, ap2, bp1, bp2):
    x1, y1 = ap1
    x2, y2 = ap2
    x3, y3 = bp1
    x4, y4 = bp2
    
    A = (x1 - x2)*(y3 - y4) - (y1 - y2)*(x3 - x4)
    
    if A != 0:
        t = (x1 - x3)*(y3 - y4) - (y1 - y3)*(x3 - x4)
        t /= A
            
        ix = x1 + t*(x2 - x1)
        iy = y1 + t*(y2 - y1)

    return Point(ix, iy)

state = dict()
img = np.zeros((h,w,3), np.uint8)

pos = dict()
pos['ball'] = Point(0, 0)
pos['pad'] = Point(0, 0)

previous = dict()
previous['ball'] = Point(1.0, 1.0)
intersect_point = Point(0, 0)

def update_from_output(output_deque, img):
    while output_deque:
        x = output_deque.popleft()
        y = output_deque.popleft()
        t = output_deque.popleft()
        
        if t == 3:
            pos['pad'] = Point(x, y)
        if t == 4:
            previous['ball'] = Point(pos['ball'].x, pos['ball'].y)
            pos['ball'] = Point(x, y)
            
        
        state[Point(x, y)] = t
        
        if x != -1:
            img[y, x] = colors[t]

p = Prog(prog, exit_on_output=False, exit_on_input=True)
p.memory[0] = 2
p.run()
p.exit_on_output = True

last_score = -1

while True:
    exit_code = p.run()
    if len(p.output_deque) > 1 and len(p.output_deque) % 3 == 0:
        update_from_output(p.output_deque, img)
        score = state[Point(-1, 0)]
        if score != last_score:
            print('New score', score,' Blocks remaining', len([p for p in state.values() if p == 2]))
            last_score = score
            if len([p for p in state.values() if p == 2]) == 0:
                break
 
        pos['inter'] = intersect(previous['ball'], pos['ball'], Point(1, 21), Point(41, 21))
        if pos['ball'].x > pos['pad'].x:
            p.input_deque = deque([1])
        elif pos['ball'].x < pos['pad'].x:
            p.input_deque = deque([-1])
        else:
            p.input_deque = deque([0])

print('Final score {}'.format(state[Point(-1, 0)]))
    


In [ ]:
from collections import defaultdict, deque

def opc(s):
    s_ = '{:05d}'.format(int(s))
    code = int(s_[-2:])
    params = s_[0:3]
    return code, params[::-1]

class Prog:
    def __init__(self, prog, exit_on_output=False, exit_on_input=False):
        self.prog = prog.copy()
        self.base_prog = prog.copy()
        self.offset = 0
        self.input_deque = deque()
        self.output_deque = deque()
        self.finished = False
        self.memory = defaultdict(lambda: 0)
        self.rel_base = 0
        self.exit_on_output = exit_on_output
        self.exit_on_input = exit_on_input
        
    def run(self):
        _continue = True
#         print('Run with ', self.input_deque)
        while _continue:
            _continue, _offset, exit_code = self.op2(self.prog)
            self.offset += _offset
        return exit_code
            
    def mem_read(self, s, addr, relative=False):
        if addr < len(s):
            return s[addr]
        else:
            return self.memory[addr]
    
    def mem_write(self, s, addr_, val, relative=False):
        if relative:
            addr = self.rel_base + addr_
        else:
            addr = addr_
        if addr < len(s):
            s[addr] = val
        else:
            self.memory[addr] = val
        
    def op2(self, s):
        offset = self.offset
        code, params = opc(s[offset])
#         print(s[offset], code, offset, params)
        _off = 4
        if code == 3 or code == 4 or code == 9:
            _off = 2
        if code == 5 or code == 6:
            _off = 3

        if code == 99:
            self.finished = True
            return False, 1, 99
        pr = s[offset:offset+_off].copy()

        ppr = []
        for i, val in enumerate(pr[1:]):
    #         print(i, val)
            if int(params[i]) == 0:
                addr = int(val)
                ppr.append(int(self.mem_read(s, addr)))
            elif int(params[i]) == 1:
                ppr.append(int(val))
            else:
                addr = self.rel_base + int(val)
                ppr.append(int(self.mem_read(s, addr)))

    #     print(ppr)

        if code == 3:
    #         in_ = input()
            in_ = self.input_deque.popleft()
            in_ = int(in_)
            self.mem_write(s, int(pr[-1]), in_, int(params[0])==2)
            return (not self.exit_on_input), _off, code
        elif code == 4:
#             print('>>>', s[int(pr[-1])])
            self.output_deque.append(ppr[0])
            return (not self.exit_on_output), _off, code
        elif code == 5:
            flag = ppr[0]
            if flag > 0:
                new_addr = ppr[1]
                return True, new_addr - offset, code
            else:
                return True, _off, code
        elif code == 6:
            flag = ppr[0]
            if flag == 0:
                new_addr = ppr[1]
                return True, new_addr - offset, code
            else:
                return True, _off, code
        elif code == 9:
            self.rel_base += ppr[0]
            return True, _off, code

        if code == 1:
            self.mem_write(s,int(pr[-1]), ppr[0] + ppr[1], int(params[-1])==2)
            return True, _off, code
        elif code == 2:
            self.mem_write(s,int(pr[-1]), ppr[0] * ppr[1], int(params[-1])==2)
            return True, _off, code
        elif code == 7:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] < ppr[1] else 0, int(params[-1])==2)
            return True, _off, code
        elif code == 8:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] == ppr[1] else 0, int(params[-1])==2)
            return True, _off, code
        elif code == 99:
            return False, 1, code
        else:
            print(code)
            print('Error')
            return False, 0, -1

## P12
### Part 1

In [ ]:
import re

class Planet:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0
        self.vx = 0
        self.vy = 0
        self.vz = 0

planets = []
with open('p12_input.txt') as f:
    for line in f:
        p = Planet()
        elems = re.split("[=,]", line.strip()[1:-1])
        for i in range(3):
            setattr(p, elems[2*i].strip(), int(elems[2*i+1]))
        planets.append(p)

In [ ]:
def state(p):
    return hash((p[0].x, p[0].y, p[0].z,
         p[0].vx, p[0].vy, p[0].vz, 
         p[1].x, p[1].y, p[1].z,
         p[1].vx, p[1].vy, p[1].vz, 
         p[2].x, p[2].y, p[2].z,
         p[2].vx, p[2].vy, p[2].vz,
         p[3].x, p[3].y, p[3].z,
         p[3].vx, p[3].vy, p[3].vz))
    

for i in range(1000):
    for p1 in planets:
        dx = dy = dz = 0
        for p2 in planets:
            if p1.x > p2.x:
                dx -= 1
            elif p1.x < p2.x:
                dx += 1
            if p1.y > p2.y:
                dy -= 1
            elif p1.y < p2.y:
                dy += 1
            if p1.z > p2.z:
                dz -= 1
            elif p1.z < p2.z:
                dz += 1
        p1.vx += dx
        p1.vy += dy
        p1.vz += dz

    energy = 0
    for p in planets:
        p.x += p.vx
        p.y += p.vy
        p.z += p.vz
        
#         for attr in ['x', 'y', 'z', 'vx', 'vy', 'vz']:
#             print(attr, getattr(p, attr),end=' ')
#         print()

        energy += sum(abs(getattr(p, attr)) for attr in ['x', 'y', 'z'])*sum(abs(getattr(p, attr)) for attr in ['vx', 'vy', 'vz'])
    
print(energy)

## P11
### Part 1

In [ ]:
with open('p11_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
prog = [3,8,1005,8,309,1106,0,11,0,0,0,104,1,104,0,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,1,10,4,10,1001,8,0,29,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,51,3,8,102,-1,8,10,1001,10,1,10,4,10,108,0,8,10,4,10,1002,8,1,72,1,1104,8,10,2,1105,15,10,2,1106,0,10,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,1,10,4,10,101,0,8,107,3,8,102,-1,8,10,1001,10,1,10,4,10,108,1,8,10,4,10,101,0,8,128,2,6,8,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,155,1006,0,96,2,108,10,10,1,101,4,10,3,8,1002,8,-1,10,101,1,10,10,4,10,1008,8,0,10,4,10,1002,8,1,188,2,1,5,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,214,2,6,18,10,1006,0,78,1,105,1,10,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,1,10,4,10,102,1,8,247,2,103,8,10,2,1002,10,10,2,106,17,10,1,1006,15,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,1,10,4,10,101,0,8,285,1,1101,18,10,101,1,9,9,1007,9,992,10,1005,10,15,99,109,631,104,0,104,1,21102,387507921664,1,1,21102,1,326,0,1106,0,430,21102,932826591260,1,1,21102,337,1,0,1106,0,430,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,21101,206400850983,0,1,21101,0,384,0,1105,1,430,21102,3224464603,1,1,21102,395,1,0,1106,0,430,3,10,104,0,104,0,3,10,104,0,104,0,21102,838433657700,1,1,21102,418,1,0,1106,0,430,21101,825012007272,0,1,21101,429,0,0,1106,0,430,99,109,2,21202,-1,1,1,21101,40,0,2,21101,461,0,3,21102,1,451,0,1105,1,494,109,-2,2105,1,0,0,1,0,0,1,109,2,3,10,204,-1,1001,456,457,472,4,0,1001,456,1,456,108,4,456,10,1006,10,488,1102,1,0,456,109,-2,2106,0,0,0,109,4,1202,-1,1,493,1207,-3,0,10,1006,10,511,21101,0,0,-3,21202,-3,1,1,21201,-2,0,2,21102,1,1,3,21102,1,530,0,1106,0,535,109,-4,2106,0,0,109,5,1207,-3,1,10,1006,10,558,2207,-4,-2,10,1006,10,558,22101,0,-4,-4,1106,0,626,22102,1,-4,1,21201,-3,-1,2,21202,-2,2,3,21101,0,577,0,1106,0,535,22102,1,1,-4,21101,1,0,-1,2207,-4,-2,10,1006,10,596,21102,0,1,-1,22202,-2,-1,-2,2107,0,-3,10,1006,10,618,21201,-1,0,1,21102,618,1,0,105,1,493,21202,-2,-1,-2,22201,-4,-2,-4,109,-5,2105,1,0]

In [ ]:
from collections import defaultdict, namedtuple

def get_new_orientation(orientation, vector):
    if vector == 0:
        #left
        if orientation == 'up':
            return 'left'
        elif orientation == 'left':
            return 'down'
        elif orientation == 'down':
            return 'right'
        elif orientation == 'right':
            return 'up'
    else:
        if orientation == 'up':
            return 'right'
        elif orientation == 'left':
            return 'up'
        elif orientation == 'down':
            return 'left'
        elif orientation == 'right':
            return 'down'
        
        
def get_new_location(loc, orientation):
    nx, ny = loc
    if orientation == 'up':
        ny += 1
    elif orientation == 'left':
        nx -= 1
    elif orientation == 'down':
        ny -= 1
    elif orientation == 'right':
        nx += 1
    return Point(nx, ny)

Point = namedtuple('Point', ['x', 'y']) 

current_location = Point(0, 0)

paint_map = defaultdict(lambda: 0)

orientation = 'up'

p = Prog(prog)

while not p.finished:
    p.input_deque.append(paint_map[current_location])
    p.run()
    if p.output_deque:
        paint_map[current_location] = p.output_deque.pop()
    p.run()
    if p.output_deque:
        vector = p.output_deque.pop()
        orientation = get_new_orientation(orientation, vector)
        current_location = get_new_location(current_location, orientation)
    
len(paint_map.keys())

# while not p.finished:
    
    

# def run(p):
#     while not p.finished:
#         p.run()
#     print(p.output_deque)
    
# run(p)

### Part 2

In [ ]:
current_location = Point(0, 0)

paint_map = defaultdict(lambda: 0)
paint_map[current_location] = 1

orientation = 'up'

p = Prog(prog)

while not p.finished:
    p.input_deque.append(paint_map[current_location])
    p.run()
    if p.output_deque:
        paint_map[current_location] = p.output_deque.pop()
    p.run()
    if p.output_deque:
        vector = p.output_deque.pop()
        orientation = get_new_orientation(orientation, vector)
        current_location = get_new_location(current_location, orientation)

In [ ]:
min_x = min([x for x, y in paint_map.keys()])
min_y = min([y for x, y in paint_map.keys()])
max_x = max([x for x, y in paint_map.keys()])
max_y = max([y for x, y in paint_map.keys()])

print(min_x, min_y, max_x, max_y)

w = max_x - min_x
h = max_y - min_y

import numpy as n
import matplotlib
import matplotlib.pyplot as plt

np_img = np.zeros((h,w,3), np.uint8)

ptr = 0
for hp in range(h):
    for wp in range(w):
        if paint_map[Point(wp + min_x, hp + min_y)] == 1:
            np_img[max_y - hp, wp] = (255, 255, 255)
        ptr += 1

matplotlib.pyplot.imshow(np_img)
# APUGURFH

In [ ]:
from collections import defaultdict, deque

def opc(s):
    s_ = '{:05d}'.format(int(s))
    code = int(s_[-2:])
    params = s_[0:3]
    return code, params[::-1]

class Prog:
    def __init__(self, prog):
        self.prog = prog.copy()
        self.base_prog = prog.copy()
        self.offset = 0
        self.input_deque = deque()
        self.output_deque = deque()
        self.finished = False
        self.memory = defaultdict(lambda: 0)
        self.rel_base = 0
        
    def run(self):
        _continue = True
#         print('Run with ', self.input_deque)
        while _continue:
            _continue, _offset = self.op2(self.prog)
            self.offset += _offset
            
    def mem_read(self, s, addr, relative=False):
        if addr < len(s):
            return s[addr]
        else:
            return self.memory[addr]
    
    def mem_write(self, s, addr_, val, relative=False):
        if relative:
            addr = self.rel_base + addr_
        else:
            addr = addr_
        if addr < len(s):
            s[addr] = val
        else:
            self.memory[addr] = val
        
    def op2(self, s):
        offset = self.offset
        code, params = opc(s[offset])
#         print(s[offset], code, offset, params)
        _off = 4
        if code == 3 or code == 4 or code == 9:
            _off = 2
        if code == 5 or code == 6:
            _off = 3

        if code == 99:
            self.finished = True
            return False, 1
        pr = s[offset:offset+_off].copy()

        ppr = []
        for i, val in enumerate(pr[1:]):
    #         print(i, val)
            if int(params[i]) == 0:
                addr = int(val)
                ppr.append(int(self.mem_read(s, addr)))
            elif int(params[i]) == 1:
                ppr.append(int(val))
            else:
                addr = self.rel_base + int(val)
                ppr.append(int(self.mem_read(s, addr)))

    #     print(ppr)

        if code == 3:
    #         in_ = input()
            in_ = self.input_deque.popleft()
            in_ = int(in_)
            self.mem_write(s, int(pr[-1]), in_, int(params[0])==2)
            return True, _off
        elif code == 4:
#             print('>>>', s[int(pr[-1])])
            self.output_deque.append(ppr[0])
            return False, _off
        elif code == 5:
            flag = ppr[0]
            if flag > 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 6:
            flag = ppr[0]
            if flag == 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 9:
            self.rel_base += ppr[0]
            return True, _off

        if code == 1:
            self.mem_write(s,int(pr[-1]), ppr[0] + ppr[1], int(params[-1])==2)
            return True, _off
        elif code == 2:
            self.mem_write(s,int(pr[-1]), ppr[0] * ppr[1], int(params[-1])==2)
            return True, _off
        elif code == 7:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] < ppr[1] else 0, int(params[-1])==2)
            return True, _off
        elif code == 8:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] == ppr[1] else 0, int(params[-1])==2)
            return True, _off
        elif code == 99:
            return False, 1
        else:
            print('Error')
            return False

## P10
### Part 1

In [ ]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y']) 

belt = set()
with open('p10_input.txt') as f:
    for y, line in enumerate(f):
        for x, c in enumerate(line):
            if c == '#':
                belt.add(Point(x, y))
                
H, W = y, x

In [ ]:
def los(p1, p2):
    l = []
    sx, sy = p1
    lx, ly = p2
    if abs(lx - sx) == 0:
        my = min(sy, ly)
        ly = max(sy, ly)
        for dy in range(ly - my + 1):
            l.append(Point(sx, my + dy))
    else:
        m = (ly - sy) / abs(lx - sx)
        for dx in range(abs(lx - sx) + 1):
            nx = sx+dx if lx > sx else sx - dx
            l.append(Point(nx, sy + m*dx))
    return l
    
scores = []
for src in belt:
    cnt = 0
    for dst in belt - {src}:
        if not set(los(src, dst)) & belt - {src, dst}:
            cnt += 1
    scores.append((src, cnt))

best_loc, best_score = max(scores, key=lambda x: x[1])
print(best_score, best_loc)

### Part 2

In [ ]:
def polar(c, x):
    dx = (c.x - x.x)
    dy = (c.y - x.y)
    r = math.sqrt(dx**2 + dy**2)
    return (180*math.atan2(-dy, -dx)/math.pi), r

def polar_map(c, x):
    theta, r = polar(c, x)
    theta += 90.0
    theta = 360.0 + theta if theta < 0 else theta
    return theta, r

polar_belt = [(*polar_map(best_loc, ast), ast) for ast in belt - {best_loc}]
polar_belt.sort(key=lambda x: x[0])

In [ ]:
polar_belt.sort(key=lambda x: x[0])

from collections import defaultdict

stack = defaultdict(list)
for theta, r, ast in polar_belt:
    stack[theta].append((r, ast))
    
for asteroids in stack.values():
    asteroids.sort(key=lambda t: t[0])
    
skeys = list(stack.keys())
skeys.sort()
count = 0
stop = False
while not stop:
    for key in skeys:
        if stack[key]:
            d, ast = stack[key].pop(0)
            count += 1
        if count == 200:
            print(ast)
            stop = True

## P9

In [ ]:
with open('p9_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
prog1 = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
prog2 = [1102,34915192,34915192,7,4,7,99,0]
prog3 = [104,1125899906842624,99]

p1 = Prog(prog1)
p2 = Prog(prog2)
p3 = Prog(prog3)

p = Prog(prog)
p.input_deque.append(1)

def run(p):
    while not p.finished:
        p.run()
    print(p.output_deque)
    
run(p)

In [ ]:
p = Prog(prog)
p.input_deque.append(2)
run(p)

In [ ]:
from collections import defaultdict, deque

def opc(s):
    s_ = '{:05d}'.format(int(s))
    code = int(s_[-2:])
    params = s_[0:3]
    return code, params[::-1]

class Prog:
    def __init__(self, prog):
        self.prog = prog.copy()
        self.base_prog = prog.copy()
        self.offset = 0
        self.input_deque = deque()
        self.output_deque = deque()
        self.finished = False
        self.memory = defaultdict(lambda: 0)
        self.rel_base = 0
        
    def run(self):
        _continue = True
#         print('Run with ', self.input_deque)
        while _continue:
            _continue, _offset = self.op2(self.prog)
            self.offset += _offset
            
    def mem_read(self, s, addr, relative=False):
        if addr < len(s):
            return s[addr]
        else:
            return self.memory[addr]
    
    def mem_write(self, s, addr_, val, relative=False):
        if relative:
            addr = self.rel_base + addr_
        else:
            addr = addr_
        if addr < len(s):
            s[addr] = val
        else:
            self.memory[addr] = val
        
    def op2(self, s):
        offset = self.offset
        code, params = opc(s[offset])
#         print(s[offset], code, offset, params)
        _off = 4
        if code == 3 or code == 4 or code == 9:
            _off = 2
        if code == 5 or code == 6:
            _off = 3

        if code == 99:
            self.finished = True
            return False, 1
        pr = s[offset:offset+_off].copy()

        ppr = []
        for i, val in enumerate(pr[1:]):
    #         print(i, val)
            if int(params[i]) == 0:
                addr = int(val)
                ppr.append(int(self.mem_read(s, addr)))
            elif int(params[i]) == 1:
                ppr.append(int(val))
            else:
                addr = self.rel_base + int(val)
                ppr.append(int(self.mem_read(s, addr)))

    #     print(ppr)

        if code == 3:
    #         in_ = input()
            in_ = self.input_deque.popleft()
            in_ = int(in_)
            self.mem_write(s, int(pr[-1]), in_, int(params[0])==2)
            return True, _off
        elif code == 4:
#             print('>>>', s[int(pr[-1])])
            self.output_deque.append(ppr[0])
            return False, _off
        elif code == 5:
            flag = ppr[0]
            if flag > 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 6:
            flag = ppr[0]
            if flag == 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 9:
            self.rel_base += ppr[0]
            return True, _off

        if code == 1:
            self.mem_write(s,int(pr[-1]), ppr[0] + ppr[1], int(params[-1])==2)
            return True, _off
        elif code == 2:
            self.mem_write(s,int(pr[-1]), ppr[0] * ppr[1], int(params[-1])==2)
            return True, _off
        elif code == 7:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] < ppr[1] else 0, int(params[-1])==2)
            return True, _off
        elif code == 8:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] == ppr[1] else 0, int(params[-1])==2)
            return True, _off
        elif code == 99:
            return False, 1
        else:
            print('Error')
            return False

## P8

In [ ]:
with open('p8_input.txt') as f:
    img_buffer = f.read().strip()

In [ ]:
w = 25
h = 6
nl = int(len(img_buffer) / (w*h))
print('Image is {}x{}x{}'.format(w, h, nl))

In [ ]:
img = []
ptr = 0
for _ in range(nl):
    layer = []
    for _ in range(h):
        for _ in range(w):
            layer.append(img_buffer[ptr])
            ptr += 1
    img.append(layer)


In [ ]:
from collections import Counter

counters = []
for l in range(nl):
    counters.append(Counter(img[l]))
    
target = min(cnt['0'] for cnt in counters)

for l in range(nl):
    if counters[l]['0'] == target:
        print(counters[l]['1']*counters[l]['2'])

In [ ]:
acc = img[0].copy()

In [ ]:
for l in range(1, nl):
    for ptr in range(len(acc)):
        if int(acc[ptr]) == 2:
            if int(img[l][ptr]) < 2:
                acc[ptr] = img[l][ptr]

In [ ]:
def print_img(acc):
    ptr = 0
    for hp in range(h):
        for wp in range(w):
            print(acc[ptr], end='')
            ptr += 1
        print()
        
print_img(acc)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np_img = np.zeros((h,w,3), np.uint8)

ptr = 0
for hp in range(h):
    for wp in range(w):
        if int(acc[ptr]) == 1:
            np_img[hp, wp] = (255, 255, 255)
        ptr += 1

matplotlib.pyplot.imshow(np_img)

## P7

In [ ]:
prog1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
prog2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,
101,5,23,23,1,24,23,23,4,23,99,0,0]

In [ ]:
from collections import deque
inp = deque([3, 0])
inp.popleft()

In [ ]:
run_prog(prog1)

In [ ]:
with open('p7_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
params = [0, 1, 2, 3, 4]
from itertools import permutations

perms = permutations(params)

output = []

for perm in perms:
    inp = deque([0])
    for p in perm:
        inp.appendleft(p)
        prog_ = prog.copy()
        run_prog(prog_, inp)
    output.append(inp[0])
print(max(output))

In [ ]:
class Prog:
    def __init__(self, prog):
        self.prog = prog.copy()
        self.base_prog = prog.copy()
        self.offset = 0
        self.input_deque = deque()
        self.output_deque = deque()
        self.finished = False
        
    def run(self):
        _continue = True
#         print('Run with ', self.input_deque)
        while _continue:
            _continue, _offset = self.op2(self.prog)
            self.offset += _offset
        
    def op2(self, s):
        offset = self.offset
        code, params = opc(s[offset])
    #     print(s[offset], code, offset, params)
        _off = 4
        if code == 3 or code == 4:
            _off = 2
        if code == 5 or code == 6:
            _off = 3

        if code == 99:
            self.finished = True
            return False, 1
        pr = s[offset:offset+_off].copy()

        ppr = []
        for i, val in enumerate(pr[1:]):
    #         print(i, val)
            if int(params[i]) == 0:
                ppr.append(int(s[int(val)]))
            else:
                ppr.append(int(val))

    #     print(ppr)

        if code == 3:
    #         in_ = input()
            in_ = self.input_deque.popleft()
            in_ = int(in_)
            s[int(pr[-1])] = in_
            return True, _off
        elif code == 4:
#             print('>>>', s[int(pr[-1])])
            self.output_deque.append(s[int(pr[-1])])
            return False, _off
        elif code == 5:
            flag = ppr[0]
            if flag > 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 6:
            flag = ppr[0]
            if flag == 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off

        if code == 1:
            s[int(pr[-1])] = ppr[0] + ppr[1]
            return True, _off
        elif code == 2:
            s[int(pr[-1])] = ppr[0] * ppr[1]
            return True, _off
        elif code == 7:
            s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
            return True, _off
        elif code == 8:
            s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
            return True, _off
        elif code == 99:
            return False, 1
        else:
            print('Error')
            return False

In [ ]:
# prog3 = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
# 27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

# prog4 = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
# -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
# 53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]


# phase_params = [9, 8, 7, 6, 5]
# phase_params = [9, 7, 8, 5, 6]

def run_feedback(phase_params):
    amps = [Prog(prog) for i in range(5)]

    for i in range(5):
        amps[i].input_deque.append(phase_params[i])

    amps[0].input_deque.append(0)

    finished = False
    res = 0
    while not finished:
        for i in range(5):
            amps[i].run()
            next_idx = (i+1) % 5
#             print(next_idx)
            if amps[i].output_deque:
                res = amps[i].output_deque.pop()
                amps[next_idx].input_deque.append(res)
            finished = amps[i].finished
#             if finished:
#                 print('Prog ', i, ' finished')
    return res
    
params = [5, 6, 7, 8, 9]
from itertools import permutations

perms = permutations(params)

output = []

for perm in perms:
    output.append(run_feedback(perm))
print(max(output))

In [ ]:
12 % 10

In [ ]:
def op2(s, offset, inp):
    code, params = opc(s[offset])
#     print(s[offset], code, offset, params)
    _off = 4
    if code == 3 or code == 4:
        _off = 2
    if code == 5 or code == 6:
        _off = 3
    
    if code == 99:
        return False, -1
    pr = s[offset:offset+_off].copy()
    
    ppr = []
    for i, val in enumerate(pr[1:]):
#         print(i, val)
        if int(params[i]) == 0:
            ppr.append(int(s[int(val)]))
        else:
            ppr.append(int(val))
            
#     print(ppr)
    
    if code == 3:
#         in_ = input()
        in_ = inp.popleft()
        in_ = int(in_)
        s[int(pr[-1])] = in_
        return True, _off
    elif code == 4:
        print('>>>', s[int(pr[-1])])
        inp.append(s[int(pr[-1])])
        if phase_params:
            p = phase_params.popleft()
            inp.appendleft(p)
#             phase_params.append(p)
#         phase_params.append(p)
#         print(phase_params, inp)
        return False, _off
    elif code == 5:
        flag = ppr[0]
        if flag > 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    elif code == 6:
        flag = ppr[0]
        if flag == 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    
    if code == 1:
        s[int(pr[-1])] = ppr[0] + ppr[1]
        return True, _off
    elif code == 2:
        s[int(pr[-1])] = ppr[0] * ppr[1]
        return True, _off
    elif code == 7:
        s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
        return True, _off
    elif code == 8:
        s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
        return True, _off
    elif code == 99:
        return False, 1
    else:
        print('Error')
        return False

def run_prog2(prog, inp):
    offset = 0
    _continue = True
    while _continue:
        _continue, _offset = op2(prog, offset, inp)
        #print('Operation at {}'.format(offset))
        offset += _offset
    if _offset == -1:
        return True
    else:
        return False

In [ ]:
def op(s, offset, inp):
    code, params = opc(s[offset])
#     print(s[offset], code, offset, params)
    _off = 4
    if code == 3 or code == 4:
        _off = 2
    if code == 5 or code == 6:
        _off = 3
    
    if code == 99:
        return False, 1
    pr = s[offset:offset+_off].copy()
    
    ppr = []
    for i, val in enumerate(pr[1:]):
#         print(i, val)
        if int(params[i]) == 0:
            ppr.append(int(s[int(val)]))
        else:
            ppr.append(int(val))
            
#     print(ppr)
    
    if code == 3:
#         in_ = input()
        in_ = inp.popleft()
        in_ = int(in_)
        s[int(pr[-1])] = in_
        return True, _off
    elif code == 4:
        print('>>>', s[int(pr[-1])])
        inp.append(s[int(pr[-1])])
        return True, _off
    elif code == 5:
        flag = ppr[0]
        if flag > 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    elif code == 6:
        flag = ppr[0]
        if flag == 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    
    if code == 1:
        s[int(pr[-1])] = ppr[0] + ppr[1]
        return True, _off
    elif code == 2:
        s[int(pr[-1])] = ppr[0] * ppr[1]
        return True, _off
    elif code == 7:
        s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
        return True, _off
    elif code == 8:
        s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
        return True, _off
    elif code == 99:
        return False, 1
    else:
        print('Error')
        return False

def run_prog(prog, inp):
    offset = 0
    _continue = True
    while _continue:
        _continue, _offset = op(prog, offset, inp)
        #print('Operation at {}'.format(offset))
        offset += _offset

In [ ]:
with open('p6_input.txt') as f:
    orbits = f.read().split()

In [ ]:
orbits = [
'COM)B',
'B)C',
'C)D',
'D)E',
'E)F',
'B)G',
'G)H',
'D)I',
'E)J',
'J)K',
'K)L',
'K)YOU',
'I)SAN'
]

In [ ]:
from collections import defaultdict

fw = defaultdict(list)
bw = dict()

for orb in orbits:
    a, b = tuple(orb.split(')'))
    fw[a].append(b)
    bw[b] = a

In [ ]:
from collections import deque

def orbcounter(p, trace=False):
    cnt = 0
    tr = deque()
    tr.appendleft(p)
    while p != 'COM':
        cnt += 1
        p = bw[p]
        tr.appendleft(p)
    if trace:
        return cnt, tr
    return cnt

cnt = 0

for sp in bw.keys():
    p = sp
    cnt += orbcounter(p)
    
        
print(cnt)

In [ ]:
cnt1, tr1 = orbcounter('YOU', True)
cnt2, tr2 = orbcounter('SAN', True)

In [ ]:
bp = ''
for i in range(min(len(tr1), len(tr2))):
    if tr1[i] == tr2[i]:
        bp = tr1[i]
    else:
        break
    

In [ ]:
transfer = cnt1 + cnt2 - 2*(orbcounter(bp) + 1)
print(transfer)

## P5

In [ ]:
def opc(s):
    s_ = '{:05d}'.format(int(s))
    code = int(s_[-2:])
    params = s_[0:3]
    return code, params[::-1]

def op(s, offset):
    code, params = opc(s[offset])
    print(s[offset], code, offset, params)
    _off = 4
    if code == 3 or code == 4:
        _off = 2
    if code == 5 or code == 6:
        _off = 3
    
    if code == 99:
        return False, 1
    pr = s[offset:offset+_off].copy()
    
    ppr = []
    for i, val in enumerate(pr[1:]):
#         print(i, val)
        if int(params[i]) == 0:
            ppr.append(int(s[int(val)]))
        else:
            ppr.append(int(val))
            
#     print(ppr)
    
    if code == 3:
        in_ = input()
        in_ = int(in_)
        s[int(pr[-1])] = in_
        return True, _off
    elif code == 4:
        print('>>>', s[int(pr[-1])])
        return True, _off
    elif code == 5:
        flag = ppr[0]
        if flag > 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    elif code == 6:
        flag = ppr[0]
        if flag == 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    
    if code == 1:
        s[int(pr[-1])] = ppr[0] + ppr[1]
        return True, _off
    elif code == 2:
        s[int(pr[-1])] = ppr[0] * ppr[1]
        return True, _off
    elif code == 7:
        s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
        return True, _off
    elif code == 8:
        s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
        return True, _off
    elif code == 99:
        return False, 1
    else:
        print('Error')
        return False

def run_prog(prog):
    offset = 0
    _continue = True
    while _continue:
        _continue, _offset = op(prog, offset)
        #print('Operation at {}'.format(offset))
        offset += _offset

In [ ]:
with open('p5_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
run_prog(prog)

In [ ]:
prog[0:10]

In [ ]:
test = [1002,4,3,4,33]
run(test)

In [ ]:
test

In [ ]:
run_prog(prog1)

## P4

In [ ]:
low = 264360
up = 746325
cnt = 0
for x1 in range(8):
    for x2 in range(x1, 10):
        for x3 in range(x2, 10):
            for x4 in range(x3, 10):
                for x5 in range(x4, 10):
                    for x6 in range(x5, 10):
                        if not (x1 == x2 or x2 == x3 or x3 == x4 or x4 == x5 or x5 == x6):
                            continue
                        if not check_nb([x1, x2, x3, x4, x5, x6]):
                            continue
                        nb = x1 * 100000 + x2*10000 + x3*1000 + x4 *100 + x5 *10 + x6
                        if nb < low:
                            continue
                        if nb > up:
                            break
                        cnt += 1
print(cnt)

In [ ]:
from collections import Counter

def check_nb(l):
    c = Counter()
    last = l[0]
    c[last] += 1
    for i in range(1,6):
        c[l[i]] += 1
        last = l[i]
     
    return any(cnt == 2 for cnt in c.values())
check_nb([1,2,3,4,5,5])

In [ ]:
_.values()

## P3

In [ ]:
with open('p3_input.txt') as f:
    wires = f.read().split()
    wires = [w.split(',') for w in wires]
wires

In [ ]:
def create_path(w):
    path = [(0, 0)]
    x, y = (0,0)
    for d in w:
        o = d[0]
        amount = int(d[1:])
        for i in range(amount):
            if o=='R':
                x += 1
            elif o=='L':
                x -= 1
            elif o=='U':
                y -= 1
            elif o=='D':
                y += 1
            path.append((x, y))
    return path
p1 = create_path(wires[0])
p2 = create_path(wires[1])

In [ ]:
crossing_points = set(p1) & set(p2)

In [ ]:
crossing_points.remove((0, 0))

In [ ]:
min(abs(cp[0])+abs(cp[1]) for cp in crossing_points)

In [ ]:
lengths = []
for cp in crossing_points:
    l = 0
    for p in p1:
        if p == cp:
            break
        l+=1
    for p in p2:
        if p == cp:
            break
        l+=1
    lengths.append(l)
min(lengths)

## P1

In [ ]:
with open('p1_input.txt') as f:
    lines = f.read().split()
    
def fuel_need(val):
    return max(int(val / 3) - 2, 0)

def fuel_comp(val):
    tot = 0
    fn = fuel_need(val)
    while fn > 0:
        tot += fn
        fn = fuel_need(fn)
    return tot

In [ ]:
need = sum(fuel_need(int(val)) for val in lines)
print(need)

In [ ]:
need = sum(fuel_comp(int(val)) for val in lines)
print(need)

## P2

In [ ]:
def get_prog():
    return [1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,1,13,19,2,9,19,23,1,23,6,27,1,13,27,31,1,31,10,35,1,9,35,39,1,39,9,43,2,6,43,47,1,47,5,51,2,10,51,55,1,6,55,59,2,13,59,63,2,13,63,67,1,6,67,71,1,71,5,75,2,75,6,79,1,5,79,83,1,83,6,87,2,10,87,91,1,9,91,95,1,6,95,99,1,99,6,103,2,103,9,107,2,107,10,111,1,5,111,115,1,115,6,119,2,6,119,123,1,10,123,127,1,127,5,131,1,131,2,135,1,135,5,0,99,2,0,14,0]

In [ ]:
prog[1] = 12
prog[2] = 2

In [ ]:
def op(s, offset):
    if s[offset] == 99:
        return False
    pr = s[offset:offset+4].copy()
    opc = pr[0]
    a = s[pr[1]]
    b = s[pr[2]]
    if opc==1:
        s[pr[3]] = a + b
        return True
    elif opc==2:
        s[pr[3]] = a*b
        return True
    elif opc==99:
        return False
    else:
        print('Error')
        return False

def run(prog):
    offset = 0
    while op(prog, offset):
        #print('Operation at {}'.format(offset))
        offset += 4

In [ ]:
prog

In [ ]:
test1 = [1,0,0,0,99]

In [ ]:
run(test1)

In [ ]:
run(prog)

In [ ]:
prog

In [ ]:
for a in range(100):
    for b in range(100):
        p = get_prog()
        p[1] = a
        p[2] = b
        run(p)
        if p[0] == 19690720:
            print(a, b)
            break

## Misc

In [ ]:
def intersect(ap1, ap2, bp1, bp2):
    x1, y1 = ap1
    x2, y2 = ap2
    x3, y3 = bp1
    x4, y4 = bp2
    
    A = (x1 - x2)*(y3 - y4) - (y1 - y2)*(x3 - x4)
    
    if A != 0:
        t = (x1 - x3)*(y3 - y4) - (y1 - y3)*(x3 - x4)
        t /= A
            
        ix = x1 + t*(x2 - x1)
        iy = y1 + t*(y2 - y1)

    return Point(ix, iy)

intersect(Point(0, 0), Point(4, 0), Point(2, 2), Point(2, 4))